In [ ]:
# Reload all modules every time before executing the Python code typed
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import datetime
import io
import pickle
import re
import json
import cProfile
from tqdm import tqdm
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from shapely.geometry import Point, Polygon, MultiPolygon, box
import geopandas as geopd
import pandas as pd
import querier as qr
from dotenv import load_dotenv
load_dotenv()

import ses_ling.data.socioeconomic as ses_data
import ses_ling.data.access as data_access
import ses_ling.utils.paths as path_utils
import ses_ling.utils.geometry as geo_utils
import ses_ling.utils.spatial_agg as spatial_agg
import ses_ling.utils.text_process as text_process
import ses_ling.data.user_residence as user_residence
from ses_ling.language import Language
from ses_ling.region import Region

In [ ]:
paths = path_utils.ProjectPaths()
all_cntr_shapes = geopd.read_file(paths.countries_shapefile)

In [ ]:
with open(paths.ext_data / 'countries.json') as f:
    countries_dict = json.load(f)
cc = 'GB'
cc_dict = countries_dict[cc]
res_cell_size = 'MSOA_BGC'
year_from = 2015
year_to = 2021
res_attr_kwargs = dict(
    nighttime_acty_th=0.5,
    all_acty_th=0.1,
    count_th=3,
    gps_dups_th=500,
    pois_dups_th=500,
)
reg = Region(
    cc, 'en', cc_dict, res_cell_size, cell_size=res_cell_size,
    year_from=year_from, year_to=year_to,
    res_attr_kwargs=res_attr_kwargs,
)

shape_geodf loaded for GB
ses_df loaded for GB
cell_levels_corr loaded for GB
lt_rules loaded for GB


# Explore census files

## Income after housing

In [47]:
output_fname = "msoaallestimatesnetequivafterhousingcosts_MSOA"
netincome_df = pd.read_csv(paths.ext_data / f"{output_fname}_mean.csv")
for stat in ['median', 'p10', 'p25', 'p75', 'p90']:
    stat_df = pd.read_csv(paths.ext_data / f"{output_fname}_{stat}.csv")
    cols_to_keep = stat_df.columns.difference(netincome_df.columns)
    netincome_df = pd.concat([netincome_df, stat_df[cols_to_keep]], axis=1)

In [52]:
netincome_df.to_csv(paths.ext_data / f"{output_fname}.csv")

## English proficiency

In [5]:
census_table = pd.read_csv(paths.ext_data / 'msoa_english_proficiency.csv', index_col='geography code')

In [6]:
count_cols = ['totpop', 'main', 'very_well', 'well', 'not_well', 'cannot']
nr_vdims = len(count_cols)
census_table = census_table.rename(columns={old: new for old, new in zip(census_table.columns[-nr_vdims:], count_cols)})
census_table = census_table.assign(**{c: census_table[c] / census_table['totpop'] for c in count_cols[1:]})

In [11]:
for s in np.arange(1, nr_vdims - 2):
    cats = count_cols[-nr_vdims+1:]
    grp = cats[s:]
    census_table = census_table.eval(f"{grp[-1]}_to_{grp[0]} = {' + '.join(grp)}")

In [12]:
census_table = census_table[census_table.columns[-nr_vdims+2:]].add_prefix('speak_')
census_table.head()

,speak_cannot_to_main,speak_cannot_to_very_well,speak_cannot_to_well,speak_cannot_to_not_well
geography code,,,,
E02002559,1.0,0.008586,0.003764,0.001882
E02002560,1.0,0.009839,0.005697,0.001899
E02002561,1.0,0.014044,0.008642,0.003781
E02002562,1.0,0.034167,0.026057,0.009836
E02002563,1.0,0.011492,0.006029,0.002826


In [84]:
census_table.to_csv(paths.ext_data / 'msoa_nomis_agg_tables.csv')

## Education

In [14]:
census_table = pd.read_csv(paths.ext_data / 'msoa_qualifications.csv', index_col='geography code')

In [15]:
census_table.head()

,date,geography,Qualifications: All categories: Highest level of qualification; measures: Value,Qualifications: No qualifications; measures: Value,Qualifications: Highest level of qualification: Level 1 qualifications; measures: Value,Qualifications: Highest level of qualification: Level 2 qualifications; measures: Value,Qualifications: Highest level of qualification: Apprenticeship; measures: Value,Qualifications: Highest level of qualification: Level 3 qualifications; measures: Value,Qualifications: Highest level of qualification: Level 4 qualifications and above; measures: Value,Qualifications: Highest level of qualification: Other qualifications; measures: Value,Qualifications: Schoolchildren and full-time students: Age 16 to 17; measures: Value,Qualifications: Schoolchildren and full-time students: Age 18 and over; measures: Value,Qualifications: Full-time students: Age 18 to 74: Economically active: In employment; measures: Value,Qualifications: Full-time students: Age 18 to 74: Economically active: Unemployed; measures: Value,Qualifications: Full-time students: Age 18 to 74: Economically inactive; measures: Value
geography code,,,,,,,,,,,,,,,
E02002559,2011,Darlington 001,7019,976,847,1114,345,939,2572,226,198,159,76,9,74
E02002560,2011,Darlington 002,4826,1097,719,859,329,698,932,192,153,122,63,5,54
E02002561,2011,Darlington 003,4789,1060,662,748,318,698,1107,196,118,123,58,5,59
E02002562,2011,Darlington 004,4938,1511,726,816,245,688,710,242,128,142,73,12,57
E02002563,2011,Darlington 005,4511,1457,615,670,282,517,766,204,107,104,42,16,45


In [18]:
college_entry_level_cols = [
    "Qualifications: Highest level of qualification: Level 4 qualifications and above; measures: Value",
    "Qualifications: Highest level of qualification: Level 3 qualifications; measures: Value"
]
tot_col = "Qualifications: All categories: Highest level of qualification; measures: Value"
edu_df = pd.DataFrame({'prop_college_entry_level': sum([census_table[c] for c in college_entry_level_cols]) / census_table[tot_col]}, index=census_table.index)

In [21]:
edu_df.max()

prop_college_entry_level    0.866214
dtype: float64

In [22]:
edu_df.to_csv(paths.ext_data / 'msoa_college.csv')